In [6]:
# A_Aug_Fill_Flag_Class_with_BG_Fill.jpynb
# Variant of this for simply filling a class id with sampled background colour 

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt


def create_filename(file_name, source_class_id):
    base_name, ext = os.path.splitext(file_name)
    new_name = (f"{base_name}{ext}")
    #new_name = (f"Fill_CLS_{source_class_id}_{base_name}{ext}")
    return new_name

def visualize_pipeline_steps(input_folder, output_folder, source_image_name, filled_image_name):
    source_path = os.path.join(input_folder, source_image_name)
    filled_path = os.path.join(output_folder, filled_image_name)
    
    source_img = cv2.imread(source_path)
    filled_img = cv2.imread(filled_path)
    
    if source_img is None or filled_img is None:
        print(f"Error: Missing images for visualization. Source: {source_path}, Filled: {filled_path}")
        return
    
    images = [source_img, filled_img]
    titles = ["1. Source Image", "2. Background Filled"]
    
    plt.figure(figsize=(12, 8))
    for i, (img, title) in enumerate(zip(images, titles)):
        plt.subplot(1, 2, i + 1)
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.title(title)
        plt.axis("off")
    
    output_path = os.path.join(output_folder, f"Pipeline_Visualization_{filled_image_name}")
    plt.tight_layout()
    plt.savefig(output_path)
    plt.close()
    print(f"Visualization saved at: {output_path}")

def fill_target_bbox_with_background(tgt_image, tgt_annotations, target_class, padding=(10, 10, 10, 10), x_offset=0):
    top_pad, right_pad, bottom_pad, left_pad = padding
    image_height, image_width = tgt_image.shape[:2]

    modified = False
    for annotation in tgt_annotations:
        class_id, x_center, y_center, width, height = annotation
        if int(class_id) == target_class:
            x_min = int((x_center - width / 2) * image_width)
            y_min = int((y_center - height / 2) * image_height)
            x_max = int((x_center + width / 2) * image_width)
            y_max = int((y_center + height / 2) * image_height)

            x_min = max(0, x_min - left_pad)
            y_min = max(0, y_min - top_pad)
            x_max = min(image_width, x_max + right_pad)
            y_max = min(image_height, y_max + bottom_pad)

            sampled_colors = []
            regions = [
                tgt_image[max(0, y_min - top_pad):y_min, x_min:x_max],  # Top
                tgt_image[y_min:y_max, max(0, x_min - left_pad):x_min],  # Left
                tgt_image[y_min:y_max, x_max:min(image_width, x_max + right_pad)],  # Right
            ]

            for region in regions:
                if region.size > 0:
                    sampled_colors.append(region.mean(axis=(0, 1)))

            if sampled_colors:
                background_color = np.mean(sampled_colors, axis=0).astype(np.uint8)
                tgt_image[y_min:y_max, x_min:x_max] = background_color
                modified = True
            else:
                print(f"Failed to sample colors for bbox: ({x_min}, {y_min}, {x_max}, {y_max})")
            break

    return tgt_image, modified
    

def remove_object_by_bb(src_folder, output_folder, source_class_id, padding=(10, 10, 10, 10), x_offset=0): 
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    count_processed = 0
    for file_name in os.listdir(src_folder):
        if file_name.lower().endswith('.png'):
            src_image_path = os.path.join(src_folder, file_name)
            src_annotation_path = os.path.join(src_folder, os.path.splitext(file_name)[0] + ".txt")

            src_image = cv2.imread(src_image_path)
            if src_image is None or not os.path.exists(src_annotation_path):
                print(f"Skipping {file_name}: Missing image or annotations.")
                continue

            with open(src_annotation_path, 'r') as f:
                src_annotations = [list(map(float, line.split())) for line in f]

            filled_image, modified = fill_target_bbox_with_background(
                tgt_image=src_image,
                tgt_annotations=src_annotations,
                target_class=source_class_id,
                padding=padding,
                x_offset=x_offset
            )

            if not modified:
                continue

            new_filename = create_filename(file_name, source_class_id)
            output_path = os.path.join(output_folder, new_filename)
            cv2.imwrite(output_path, filled_image)

            tgt_annotations = [annotation for annotation in src_annotations if int(annotation[0]) != source_class_id]
            annotation_path = os.path.splitext(output_path)[0] + ".txt"
            with open(annotation_path, 'w') as f:
                for annotation in tgt_annotations:
                    formatted_annotation = [int(annotation[0])] + [f"{coord:.6f}" for coord in annotation[1:]]
                    f.write(' '.join(map(str, formatted_annotation)) + '\n')

            if count_processed == 0:
                visualize_pipeline_steps(src_folder, output_folder, file_name, new_filename)
            count_processed += 1


fill_class_fldr = "D:/FlagDetectionDatasets/Augmentation/Switch_class_in_images_sets/Switch_class_1_from_43_into_88/Switch_flag_into_88"
output_fldr = "D:/FlagDetectionDatasets/Augmentation/Switch_class_in_images_sets/Switch_class_1_from_43_into_88/Merge_43_to_88_with_fillB"
class_to_replace = 0  # Target class that will be replaced 
x_offset = 10         # Horizontal offset for overlay placement - fine tune differently 
#edge_blend_width = 20  # Width of edge blending
align = 'top-left'   # Alignment option ("top-left" or "top-right")
padding=(5, 5, 5, 0)   # Padding: (top, right, bottom, left) #padding=(0, 5, 5, 5)

remove_object_by_bb(fill_class_fldr, output_fldr, class_to_replace, padding, x_offset)


Visualization saved at: D:/FlagDetectionDatasets/Augmentation/Switch_class_in_images_sets/Switch_class_1_from_43_into_88/Merge_43_to_88_with_fillB\Pipeline_Visualization_frame_000000.PNG
